In [90]:
import numpy as np
from scipy.stats import bernoulli

#K groups,N arms/group, delta=confidence of success
# Bernoulli reward disrtibution
#reward distribution example 1:[[0.1,0.2,0.3,0.2],[0.6,0.6,0.6,0.6],[0.9,0.8,0.7,0.6]]
#reward distribution example 2:[[0.1,0.2,0.3,0.2],[0.1,0.1,0.1,0.1],[0.9,0.8,0.7,0.6]]--easier to distinguish

K=3;N=4;delta=1;
reward_distribution=np.array([[0.1,0.2,0.3,0.2],[0.9,0.8,0.5,0.6],[0.9,0.8,0.7,0.6]]);
pulls=np.ones((K,N));
active_groups=np.ones(K);
actual_reward_received=np.zeros((K,N));
t=1;
lcb_arms=np.zeros((K,N));ucb_arms=np.zeros((K,N));
lcb_group=np.zeros(K);ucb_group=np.zeros(K);

def eliminate_arms(active_groups,lcb_group,ucb_group):
    lcb_max=np.max(lcb_group);
    state_groups=ucb_group>=lcb_max;    
    active_groups*=state_groups;
    return active_groups
    
def calculate_bounds(lcb_group,ucb_group,lcb_arms,ucb_arms):
    lcb_arms=actual_reward_received/pulls-np.sqrt(2*np.log(t/delta)/pulls);
    ucb_arms=actual_reward_received/pulls+np.sqrt(2*np.log(t/delta)/pulls);
    lcb_group=lcb_arms.sum(axis=1);
    ucb_group=ucb_arms.sum(axis=1);
    return lcb_arms,ucb_arms,lcb_group,ucb_group;


while np.sum(active_groups)>1 and t<=500000 :
    for i in range(len(active_groups)):
        if active_groups[i]==1:
            t=t+1;
            index=np.random.choice(range(len(reward_distribution[i])));
            pulls[i][index]+=1;
            actual_reward_received[i][index]+=bernoulli.rvs(p=reward_distribution[i][index]);

    
    lcb_arms,ucb_arms,lcb_group,ucb_group=calculate_bounds(lcb_group,ucb_group,lcb_arms,ucb_arms);
    active_groups=eliminate_arms(active_groups,lcb_group,ucb_group);
    #print(lcb_group,ucb_group,"\n",pulls)

print("Stopping Time :",t)
print("Active arm :",active_groups)


Stopping Time : 290881
Active arm : [0. 0. 1.]
